In [1]:


import resppol.resppol as rp



In [2]:
datei='/home/mschauperl/programs/resppol/resppol/tmp/butanolsybyl.mol2'


In [3]:
test=rp.Molecule(datei)

test._bonds
test._atoms


AttributeError: 'NoneType' object has no attribute '_FF'

In [ ]:
import openforcefield.topology as off
test2=off.Molecule.from_smiles("c1ccccc1")
test.add_conformer_from_mol2(datei)
test.conformers






In [ ]:
espfile='/home/mschauperl/programs/resppol/resppol/tmp/butanol/conf0/molecule0.gesp'
test.conformers[0].add_polESP(espfile, "vector")
#test.conformers[0].add_baseESP(espfile)


In [ ]:
from __future__ import print_function
from openeye import oechem

mol = oechem.OEGraphMol()
oechem.OESmilesToMol(mol, "c1cocc1")

print("atoms")
for atom in mol.GetAtoms():
    print(atom.GetAtomicNum())
print("bonds")
for bond in mol.GetBonds():
    print(bond.GetOrder())

In [ ]:
from openforcefield.typing.engines.smirnoff.forcefield import ForceField
#from openforcefield.utils import get_data_filename
import os
os.getcwd()
print("mytest2")

In [ ]:

from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField

# Create a simple molecule from SMILES and turn it into a topology.
molecule = Molecule.from_file(datei)
topology = Topology.from_molecules([molecule])

# Let's label using the smirnoff99Frosst force field
forcefield = ForceField('../tmp/BCCPOL.offxml')
# Run the molecule labeling
#|print(forcefield.get_handler('vdW').parameters)
molecule_force_list = forcefield.label_molecules(topology)




# Print out a formatted description of the parameters applied to this molecule
for mol_idx, mol_forces in enumerate(molecule_force_list):
    print(f'Forces for molecule {mol_idx}')
    for force_tag, force_dict in mol_forces.items():
        print(f"\n{force_tag}:")
        for (atom_indices, parameter) in force_dict.items():
            atomstr=''
            for idx in atom_indices:
                atomstr += '%3s' % idx
            print("atoms: %s  parameter_id: %s  smirks %s" % (atomstr, parameter.id, parameter.smirks) )

In [ ]:
import numpy as np
np.array(molecule.conformers[0]._value)[0]


In [ ]:
print(test.chemical_eq_atoms)

In [ ]:
for i, properties in enumerate(molecule_force_list[0]['Bonds'].items()):
            atom_indices, parameter=properties
            print(atom_indices)

In [ ]:

molecule_force_list[0]
for i, properties in enumerate(molecule_force_list[0]['vdW'].items()):
            atom_index, parameter=properties
            print(atom_index)

In [ ]:
from __future__ import print_function
from openeye import oechem

def find_eq_atoms(f1):
    """
    Finds all equivalent atoms in a molecule. 
    :return
    Array of pairs of equivalent atoms. 
    
    Parameters:
    ----------
        f1: string
            Path to mol2 file
    
    :TODO Include rdkit support for this function
    """
    
    #Initialize Molecules
    mol1 = oechem.OEGraphMol()
    mol2 = oechem.OEGraphMol()

    # Open Input File Stream
    ifs1 = oechem.oemolistream(f1)
    ifs2 = oechem.oemolistream(f1)

    # Read from IFS to molecules
    oechem.OEReadMol2File(ifs1,mol1)
    oechem.OEReadMol2File(ifs2,mol2)

    qmol = oechem.OEQMol()
    
    # build OEQMol from OEGRAPHMOLECULE
    oechem.OEBuildMDLQueryExpressions(qmol,mol2)
    ss2=oechem.OESubSearch(qmol)
    oechem.OEPrepareSearch(mol1, ss2)
    
    # Store the equivalent atoms
    eq_atoms=[[]for i in range(mol1.NumAtoms())]
    
    # Goes through all matches and compares the atom indeces. 
    for count, match in enumerate(ss2.Match(mol1)):
        for ma in match.GetAtoms():
            # if 2 atoms are not the same atoms
            if ma.pattern.GetIdx() != ma.target.GetIdx():
                    # and the pair is not stored yet
                    if ma.target.GetIdx() not in eq_atoms[ma.pattern.GetIdx()]:
                        # save it to the array
                        eq_atoms[ma.pattern.GetIdx()].append(ma.target.GetIdx())
    
    # goes through the array and returns pairs of equivalent atoms
    sorted_eq_atoms=[]                
    for i,ele in enumerate(eq_atoms):
        if ele!=[]:
            for element in ele:
                # Making sure we have not already covered this pair of atoms
                if [element,i] not in sorted_eq_atoms:
                    sorted_eq_atoms.append([i,element])
            
    
    return(sorted_eq_atoms)
find_eq_atoms(datei)